In [1]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install wordcloud matplotlib

     |████████████████████████████████| 173 kB 1.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import plotly.express as px

In [4]:
import os
#setting working directory

working_directory = '/Users/szemin/Documents/DAC External Project/Interactive Data Visualisations'

os.chdir(working_directory)

In [5]:
# importing sample dataset

#reading files and import data
df = pd.read_csv('/Users/szemin/Documents/DAC External Project/combined_sentiments.csv')

df.head()

,Title,Content,Site-Name,Sector,Country,Financial_Sentiments,Finbert_Sentiments,Sigma_Sentiments,Soleimanian_Sentiments,Yiyangkhost_Sentiments
0,"The three big Malaysian banks, Maybank, CIMB a...",news see whats happening bfsi sector right exc...,bfsi,Oil and Gas,Malaysia,-1,0,-1,0,0
1,This article first appeared in The Edge Malays...,bank growing increasingly cautious granting lo...,edge,Oil and Gas,Malaysia,1,-1,-1,-1,0
2,ICLG - Oil & Gas Laws and Regulations -,chapter cover common issue oil gas law regulat...,iclg,Oil and Gas,Malaysia,-1,0,-1,0,0
3,Articles & Commentaries >,iseas perspective article commentary iseas pe...,Iseas,Oil and Gas,Malaysia,0,0,-1,0,0
4,The report covers Malaysia Oil and Gas Compani...,disclaimer major player sorted particular orde...,Mordor,Oil and Gas,Malaysia,-1,0,-1,0,0


#### count

In [24]:
# creating new dataframe containing count of Sentiment models

sentiments = df.groupby(["Financial_Sentiments"], as_index = False).size()

sentiments


,Financial_Sentiments,size
0,-1,16
1,0,107
2,1,19


#### Sentiment Score by Sector

In [7]:
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Sentiments Count by Sector"),
    dcc.Dropdown(
        id='sector-dropdown',
        options=[{'label': sector, 'value': sector} for sector in df['Sector'].unique()],
        value=df['Sector'].unique()[0]
    ),
    dcc.Graph(id='sector-bar-chart')
])

# Define the callback function to update the bar chart based on dropdown selection
@app.callback(
    Output('sector-bar-chart', 'figure'),
    [Input('sector-dropdown', 'value')]
)
def update_bar_chart(selected_sector):
    # Filter the DataFrame based on the selected sector
    filtered_df = df[df['Sector'] == selected_sector]
    
    # Count the occurrences of -1, 0, and 1 in the sentiment columns
    count_df = pd.concat([filtered_df[col].value_counts().reset_index().rename(columns={'index': 'Sentiment', col: 'Count'}) 
                           for col in ['Financial_Sentiments', 'Finbert_Sentiments', 'Sigma_Sentiments']], axis=0)
    
    # Create the bar chart using Plotly Express
    fig = px.bar(count_df, x='Sentiment', y='Count', color='Sentiment', 
                 labels={'Sentiment': 'Sentiment Value', 'Count': 'Count'},
                 title=f'Sentiments Count for {selected_sector}')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

/var/folders/v6/cm8bgvgx7vq6m1sghl7nhr140000gn/T/ipykernel_53239/3994678336.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/v6/cm8bgvgx7vq6m1sghl7nhr140000gn/T/ipykernel_53239/3994678336.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


#### Analysis on Sentiment Scores

In [20]:
import plotly.express as px

# Assuming df is your DataFrame with columns: 'Sector', 'Financial_Sentiments', 'Finbert_Sentiments', 
# 'Sigma_Sentiments', 'Soleimanian_Sentiments', and 'Yiyangkhost_Sentiments'

# Calculate the mean sentiment scores for each sector and sentiment model
df_mean_sentiments = df.groupby('Sector').mean().reset_index()

# Melt the DataFrame to have sentiment models as a variable
df_melted = pd.melt(df_mean_sentiments, id_vars=['Sector'], value_vars=['Financial_Sentiments', 'Finbert_Sentiments',
                                                                       'Sigma_Sentiments', 'Soleimanian_Sentiments',
                                                                       'Yiyangkhost_Sentiments'],
                     var_name='Sentiment_Model', value_name='Mean_Sentiment_Score')

# Create an interactive grouped bar chart using Plotly
fig = px.bar(df_melted, x='Sector', y='Mean_Sentiment_Score', color='Sentiment_Model',
             title='Mean Sentiment Scores Across Sentiment Models by Sector',
             labels={'Mean_Sentiment_Score': 'Mean Sentiment Score'},
             barmode='group')

# Show the interactive chart
fig.show()

In [11]:
import pandas as pd
import plotly.graph_objects as go

# Assuming df is your DataFrame with columns: 'Sector', 'Financial_Sentiments', 'Finbert_Sentiments', 
# 'Sigma_Sentiments', 'Soleimanian_Sentiments', and 'Yiyangkhost_Sentiments'

# Calculate the mean sentiment scores for each sector and sentiment model
df_mean_sentiments = df.groupby('Sector').mean().reset_index()

# Create dropdown options for each industry
dropdown_options = [{'label': sentiment_model, 'value': sentiment_model} for sentiment_model in df_mean_sentiments.columns[1:]]

# Create a Plotly figure
fig = go.Figure()

# Add traces for each sentiment model and industry
for col in df_mean_sentiments.columns[1:]:
    fig.add_trace(go.Bar(x=df_mean_sentiments['Sector'], y=df_mean_sentiments[col], name=col))

# Create buttons list with a dictionary for each dropdown option
buttons = [{'label': option['label'], 'method': 'update', 'args': [{'visible': [col == option['label'] for col in df_mean_sentiments.columns[1:]]}]} for option in dropdown_options]

# Update dropdown menus for industries
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.5,
            xanchor='center',
            y=1.1,
            yanchor='top'
        ),
    ],
)

# Update layout and axis labels
fig.update_layout(
    title='Mean Sentiment Scores by Sentiment Model and Industry',
    xaxis_title='Industry',
    yaxis_title='Mean Sentiment Score',
)

# Show the interactive chart
fig.show()


#### Count of webscraped sitenames

In [12]:
# creating new dataframe containing count of Payment Method

df_sites = df.groupby(["Site-Name"], as_index = False).size()

df_sites


,Site-Name,size
0,CapitaLand,11
1,Edge-Prop,6
2,Iseas,1
3,Mordor,1
4,NikkeiAsia,1
...,...,...
58,tla,1
59,todayonline,1
60,uob,1
61,v-key,1


In [27]:
#creating barchart and displaying it
fig = px.bar(df_sites, x = 'Site-Name', y = 'size')
fig.show()